### Script to finetune pegasus model
source https://gist.github.com/jiahao87/50cec29725824da7ff6dd9314b53c4b3

add the path to the fine tuning data excel to filename variable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

# define the paths of the two folders
folder1_path = "/content/drive/MyDrive/NLP_project/test-data/judgement"
folder2_path = "/content/drive/MyDrive/NLP_project/test-data/summary"

# create an empty list to store the data from text files
data = []

# loop through the files in folder1 and folder2
for file1, file2 in zip(os.listdir(folder1_path), os.listdir(folder2_path)):
    # read the contents of the text files
    with open(os.path.join(folder1_path, file1), 'r') as f1, \
         open(os.path.join(folder2_path, file2), 'r') as f2:
        file1_contents = f1.read().strip()
        file2_contents = f2.read().strip()

    # append the data to the list
    data.append([file1_contents, file2_contents])

# create a DataFrame from the data
df = pd.DataFrame(data, columns=['source', 'target'])


In [ ]:
df

,source,target
0,Appeal No. 1604 of 1967.\nFrom the judgment an...,The respondent was required by the Chief Inspe...
1,N: Criminal Appeal No. 87 of 1967.\n721 Appeal...,The appellants and the respondents were owners...
2,Appeal No. 182 828 of 1975.\nAppeal by Special...,Pursuant to a scheme enacted for the benefit o...
3,Civil Appeal No. 1744 of 1968.\n(Appeal by Spe...,"Khandappa Sangappa Magdum died on June 27, 196..."
4,ivil Appeal No. 4649 of 1989.\nFrom the Judgme...,"The appellant, a partnership firm, filed its r..."
...,...,...
95,Appeal No. 1695 of 1993.\nFrom the Judgment an...,The Orissa Superior Judicial Service consisted...
96,ivil Appeals Nos.\n4582 4585 of 1989.\n125 Fro...,"The appellant, a postman, and I,is father were..."
97,ases Nos. 297 and 298 of 1951.\nAppeals under ...,The petitioners in the writ petitions were app...
98,Appeal No. 1367 of 1980.\nFrom the Judgment an...,The principles of law for determination of the...


In [ ]:
df.columns

Index(['source', 'target'], dtype='object')

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.8 MB/s eta 0:00:00


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels['input_ids'])  # len(self.labels)


def prepare_data(model_name,
                 train_texts, train_labels,
                 val_texts=None, val_labels=None,
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length = 512)
    decodings = tokenizer(labels, truncation=True, padding=True, max_length = 256)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset, tokenizer


def prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  if freeze_encoder:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

  if val_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=2,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=1,    # batch size for evaluation, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=100,                  # number of update steps before evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,            # evaluation dataset
      tokenizer=tokenizer
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=2,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      tokenizer=tokenizer
    )

  return trainer

In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00


In [ ]:
train_texts, train_labels = (list(df['source'])), (list(df['target']))

model_name = 'nsi319/legal-pegasus'
train_dataset, _, _, tokenizer = prepare_data(model_name, train_texts, train_labels)
trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,3.954600
200,3.790900


TrainOutput(global_step=200, training_loss=3.872724914550781, metrics={'train_runtime': 140.3448, 'train_samples_per_second': 1.425, 'train_steps_per_second': 1.425, 'total_flos': 288946441420800.0, 'train_loss': 3.872724914550781, 'epoch': 2.0})

In [ ]:
import os
if not os.path.exists('./ouput_model/'):
    os.makedirs('./ouput_model/')
trainer.model.save_pretrained("./ouput_model/")

In [ ]:
!zip -r ouput_model.zip ./ouput_model/

  adding: ouput_model/ (stored 0%)
  adding: ouput_model/pytorch_model.bin